In [1]:
import pandas as pd
import requests
import time

# Creating a dataframe with basic country info

In [2]:
url=r'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv' # Source: https://github.com/lukes

In [3]:
country_codes=pd.read_csv(url)

In [4]:
country_codes.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [5]:
country_codes.drop(['country-code','intermediate-region','region-code','sub-region-code','intermediate-region-code'], axis=1, inplace=True)

In [6]:
country_codes.dropna(inplace=True)
country_codes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 248
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        247 non-null    object
 1   alpha-2     247 non-null    object
 2   alpha-3     247 non-null    object
 3   iso_3166-2  247 non-null    object
 4   region      247 non-null    object
 5   sub-region  247 non-null    object
dtypes: object(6)
memory usage: 13.5+ KB


# Get indicator codes from World Bank website

In [7]:

from selenium.webdriver.chrome.options import Options
from selenium import webdriver

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')


In [8]:

driver = r"C:\Users\Alberto\Downloads/chromedriver.exe" # driver path
driver = webdriver.Chrome(driver,options = opciones)
url='https://datatopics.worldbank.org/world-development-indicators/themes/economy.html'


C:\Users\Alberto\AppData\Local\Temp\ipykernel_11180\2498371203.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options = opciones)


In [9]:
driver.get(url)

In [10]:
tabla=driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/table/tbody')
lineas=tabla.find_elements_by_tag_name('tr')
elemento=[x.find_elements_by_tag_name('td') for x in lineas]
df=pd.DataFrame(elemento)


C:\Users\Alberto\AppData\Local\Temp\ipykernel_11180\1965726154.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tabla=driver.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/table/tbody')
C:\Users\Alberto\Anaconda3\envs\clase\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")


In [11]:
df=df.dropna(axis=0)
df=df.drop(list(df)[2:], axis=1)


In [12]:
df=df.apply([lambda x: x.text])
df.columns=['indicator', 'indicator_code']
indicator_codes=df
indicator_codes.head()

,indicator,indicator_code
2,GDP (current US$),NY.GDP.MKTP.CD
3,GDP growth (annual %),NY.GDP.MKTP.KD.ZG
4,"Agriculture, value added (annual % growth)",NV.AGR.TOTL.KD.ZG
5,"Industry, value added (annual % growth)",NV.IND.TOTL.KD.ZG
6,"Manufacturing, value added (annual % growth)",NV.IND.MANF.KD.ZG


# Country data from World Bank API (2010)

In [13]:
indicators=['NY.GDP.MKTP.CD','SP.POP.TOTL','NV.AGR.TOTL.ZS','NV.IND.TOTL.ZS','NV.SRV.TOTL.ZS'] 
countries=[x for x in country_codes['alpha-2']]


In [14]:
from src.src import get_data
data=get_data(indicators,countries)
data.to_csv(r'data\macro_data.csv')



In [15]:
data[data['alpha-2']=='ES']

,NY.GDP.MKTP.CD,SP.POP.TOTL,NV.AGR.TOTL.ZS,NV.IND.TOTL.ZS,NV.SRV.TOTL.ZS,alpha-2
207,1.420722e+12,46576897.0,2.431135,23.159776,66.277341,ES


In [16]:
data[data['alpha-2']=='FR']

,NY.GDP.MKTP.CD,SP.POP.TOTL,NV.AGR.TOTL.ZS,NV.IND.TOTL.ZS,NV.SRV.TOTL.ZS,alpha-2
75,2.642610e+12,65027505.0,1.604028,17.845736,70.651971,FR


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NY.GDP.MKTP.CD  206 non-null    float64
 1   SP.POP.TOTL     214 non-null    float64
 2   NV.AGR.TOTL.ZS  189 non-null    float64
 3   NV.IND.TOTL.ZS  190 non-null    float64
 4   NV.SRV.TOTL.ZS  189 non-null    float64
 5   alpha-2         247 non-null    object 
dtypes: float64(5), object(1)
memory usage: 11.7+ KB


# CO2 Emissions

In [18]:
df=pd.read_csv(r'data\co2_emission.csv') # Downloaded form https://www.kaggle.com/datasets/yoannboyere/co2-ghg-emissionsdata?resource=download

In [19]:
df.head()

,Entity,Code,Year,Annual CO₂ emissions (tonnes )
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0


In [20]:
df=df[df['Year']==2010]
df.head()

,Entity,Code,Year,Annual CO₂ emissions (tonnes )
61,Afghanistan,AFG,2010,8.452224e+06
328,Africa,NaN,2010,1.193784e+09
413,Albania,ALB,2010,4.594656e+06
521,Algeria,DZA,2010,1.178141e+08
788,Americas (other),NaN,2010,2.253919e+09


In [21]:
df.dropna(axis=0,how='any', subset='Code',inplace=True)
df.reset_index(inplace=True)



In [22]:
df.head()

,index,Entity,Code,Year,Annual CO₂ emissions (tonnes )
0,61,Afghanistan,AFG,2010,8.452224e+06
1,413,Albania,ALB,2010,4.594656e+06
2,521,Algeria,DZA,2010,1.178141e+08
3,816,Andorra,AND,2010,5.166240e+05
4,884,Angola,AGO,2010,2.886447e+07


In [23]:
df.drop(['index','Year', 'Entity'], axis=1, inplace=True)
df.head()

,Code,Annual CO₂ emissions (tonnes )
0,AFG,8.452224e+06
1,ALB,4.594656e+06
2,DZA,1.178141e+08
3,AND,5.166240e+05
4,AGO,2.886447e+07


In [24]:
df[df['Code'].map(len)<=3]

,Code,Annual CO₂ emissions (tonnes )
0,AFG,8.452224e+06
1,ALB,4.594656e+06
2,DZA,1.178141e+08
3,AND,5.166240e+05
4,AGO,2.886447e+07
...,...,...
214,VEN,1.881793e+08
215,VNM,1.361131e+08
217,YEM,2.315186e+07
218,ZMB,2.562578e+06


In [78]:
df.rename(columns={'Code':'alpha-3'},inplace=True)

In [79]:
country_emisions=df

# Creating database

In [80]:
from pymongo import MongoClient

In [81]:
cliente=MongoClient()

In [82]:
cliente.list_database_names()

['admin', 'config', 'local']

In [83]:
db = cliente['country_db']

In [84]:
country_codes_col=db['country_codes']
country_emisions_col=db['country_emisions']
country_macro_col=db['country_macro_data']

In [85]:
country_codes_col.insert_many(country_codes.to_dict('records'))

In [86]:
country_emisions_col.insert_many(country_emisions.to_dict('records'))

In [87]:
country_macro_col.insert_many(data.to_dict('records'))